# Index JSON

Let's create a table where we can play with indexes. 

In [51]:
drop table if exists dbo.users_json_indexed;
select id = identity(int, 1, 1), json_data into dbo.users_json_indexed from dbo.users_json order by id;
alter table dbo.users_json_indexed add constraint pk__users_json_indexed primary key(id);


(4 rows affected)

Total execution time: 00:00:00.023

Let's also add some data into the created table, using existing `dbo.user_json` table

In [52]:
insert into dbo.users_json_indexed (json_data) 
select u.json_data from dbo.users_json u 
cross join sys.objects 

insert into dbo.users_json_indexed (json_data) 
values (N'{"firstName": "Joe", "lastName": "Black", "age": 42}')


(448 rows affected)

(1 row affected)

Total execution time: 00:00:00.031

In [43]:
select count(*) from dbo.users_json_indexed

(1 row affected)

Total execution time: 00:00:00.014

(No column name)
453


Expose the JSON elements we want index into columns

In [44]:
alter table dbo.users_json_indexed
add FirstName as json_value(json_data, '$.firstName')
go
alter table dbo.users_json_indexed
add LastName as json_value(json_data, '$.lastName')
go
alter table dbo.users_json_indexed
add Age as json_value(json_data, '$.age')
go

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.027

Let's take a look at the table now. FirstName, LastName and Age are both accessible via JSON and via columns

In [45]:
select * from dbo.users_json_indexed where id = 5

(1 row affected)

Total execution time: 00:00:00.019

id,json_data,FirstName,LastName,Age
5,"{ ""firstName"": ""John"", ""lastName"": ""Smith"", ""isAlive"": true, ""age"": 25, ""address"": { ""streetAddress"": ""21 2nd Street"", ""city"": ""New York"", ""state"": ""NY"", ""postalCode"": ""10021-3100"" }, ""phoneNumbers"": [ { ""type"": ""home"", ""number"": ""212 555-1234"" }, { ""type"": ""office"", ""number"": ""646 555-4567"" }, { ""type"": ""mobile"", ""number"": ""123 456-7890"" } ], ""children"": [], ""spouse"": null }",John,Smith,25


Run a query without index. It will require 41 IOPS to complete, as it needs to scan the whole table

In [ ]:
set statistics io on
select * from dbo.users_json_indexed where LastName = 'Black' 
set statistics io off


Let's create an index on the column now...

In [47]:
create nonclustered index ix1 on dbo.users_json_indexed(LastName)
go

Warning! The maximum key length for a nonclustered index is 1700 bytes. The index 'ix1' has maximum length of 8000 bytes. For some combination of large values, the insert/update operation will fail.

Total execution time: 00:00:00.014

...and now run the same query again. Only 4 IOPS! That's almost 10 times less than before!

In [48]:
set statistics io on
select * from dbo.users_json_indexed where LastName = 'Black' 
set statistics io off



(1 row affected)

Table 'users_json_indexed'. Scan count 1, logical reads 4, physical reads 0, page server reads 0, read-ahead reads 0, page server read-ahead reads 0, lob logical reads 0, lob physical reads 0, lob page server reads 0, lob read-ahead reads 0, lob page server read-ahead reads 0.

Total execution time: 00:00:00.014

id,json_data,FirstName,LastName,Age
453,"{""firstName"": ""Joe"", ""lastName"": ""Black"", ""age"": 42}",Joe,Black,42


The Azure SQL engine is smart enough to figure out that index can be used also if you access JSON directly:

In [50]:
set statistics io on
select * from dbo.users_json_indexed where json_value(json_data, '$.lastName') = 'Black' 
set statistics io off

(1 row affected)

Table 'users_json_indexed'. Scan count 1, logical reads 4, physical reads 0, page server reads 0, read-ahead reads 0, page server read-ahead reads 0, lob logical reads 0, lob physical reads 0, lob page server reads 0, lob read-ahead reads 0, lob page server read-ahead reads 0.

Total execution time: 00:00:00.014

id,json_data,FirstName,LastName,Age
453,"{""firstName"": ""Joe"", ""lastName"": ""Black"", ""age"": 42}",Joe,Black,42
